In [115]:
import urllib.request
import urllib.parse
import simplejson as json
import pandas as pd
import re

In [120]:
places_list = []
url = 'https://kudago.com/public-api/v1.2/places/?location=spb&page_size=100&fields=id,title,slug,address,site_url,foreign_url,subway,is_closed,timetable,description,body_text,coords,favorites_count,categories,tags'

In [117]:
def parse_list(url):
    user_agent = 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'
    headers = {'User-Agent': user_agent}
    req = urllib.request.Request(url, headers=headers)
    try:
        with urllib.request.urlopen(req) as response:
            the_page = response.read()
    except urllib.error.URLError as e:
        print(e.reason)

    data = json.loads(the_page)
    return data

def clear(text):
    cleared = re.sub(r"[\r\n\t]+", " ", str(text))
    return cleared

In [121]:
obj = parse_list(url)

In [122]:
while True:
    if obj['results']:
        tmp_list = []
        for i in range(len(obj['results'])):
            loc = []
            loc.append(obj['results'][i]['id'])
            loc.append(obj['results'][i]['title'])
            loc.append(obj['results'][i]['slug'])
            loc.append(obj['results'][i]['address'])
            loc.append(obj['results'][i]['timetable'])
            loc.append(clear(obj['results'][i]['body_text']))
            loc.append(clear(obj['results'][i]['description']))
            loc.append(obj['results'][i]['site_url'])
            loc.append(obj['results'][i]['foreign_url'])
            loc.append(obj['results'][i]['coords']['lat'])
            loc.append(obj['results'][i]['coords']['lon'])
            loc.append(obj['results'][i]['subway'])
            loc.append(obj['results'][i]['favorites_count'])
            loc.append(obj['results'][i]['is_closed'])
            categories = '###'.join(obj['results'][i]['categories'])
            loc.append(categories)
            tags = '###'.join(obj['results'][i]['tags'])
            loc.append(tags)
            tmp_list.append(loc)
        places_list = places_list + tmp_list
        if obj['next']:
            next1 = obj['next']
            obj = []
            obj = parse_list(next1)
        else:
            break

In [123]:
places_table = pd.DataFrame(places_list, columns=['id','title','slug','address','timetable','body_text','description', 
                                                 'site_url','foreign_url','lat','lon','subway','favorites_count', 
                                                 'is_closed', 'categories', 'tags'])

In [124]:
places_table

,id,title,slug,address,timetable,body_text,description,site_url,foreign_url,lat,lon,subway,favorites_count,is_closed,categories,tags
0,2813,Мозаичный дворик,mozaichnyj-dvorik-spb,"ул. Чайковского, д. 2",,"<p>На улице Чайковского есть маленький дворик,...",<p>Среди угрюмых питерских дворов-колодцев зат...,https://kudago.com/spb/place/mozaichnyj-dvorik...,,59.946671,30.341246,Чернышевская,1544,False,yard###kids###attractions###photo-places,дворы (интересные места)###детские площадки###...
1,918,Юсуповский дворец,yusupovskij-dvorec,"наб. реки Мойки, д. 94",ежедневно 11:00–17:00,"<p><strong>Юсуповский дворец</strong>, располо...",<p>На первых изображениях петровского Петербур...,https://kudago.com/spb/place/yusupovskij-dvorec/,http://www.yusupov-palace.ru/,59.929384,30.299043,Садовая,806,False,attractions###museums,"красиво###музеи (раздел музеи, выставки)###зам..."
2,3986,историческое шоу «Ужасы Петербурга»,uzhasy-peterburga,"ул. Марата, д. 86, ТРК «Нептун», 2 этаж (над о...","пн–чт 12:00–19:00, пт–вс 11:00–20:40","<p>Лабиринт состоит из 13-ти комнат, каждая ко...",<p>Грандиозный интерактивный музей-театр на пл...,https://kudago.com/spb/place/uzhasy-peterburga/,http://www.spbhorror.ru/,59.918300,30.338901,Пушкинская,299,False,interesting-places###museums###theatre,"шоу###интересное###музеи (раздел музеи, выстав..."
3,1060,Музей «Гранд Макет Россия»,grandmaket,"ул. Цветочная, д.16",ежедневно 10:00–20:00,<p><strong>Музей «Гранд Макет Россия»</strong>...,<p>Россия в миниатюре на площади 800 квадратны...,https://kudago.com/spb/place/grandmaket/,http://www.grandmaket.ru/,59.886772,30.330052,Московские Ворота,946,False,kids###museums,"с девушкой###музеи (раздел музеи, выставки)###..."
4,10328,«Семимостье»,semimoste,"наб. канала Грибоедова, д. 131",,<p>Если вы окажетесь на набережной рядом с Ник...,<p>Образ Петербурга неразрывно связан с мостам...,https://kudago.com/spb/place/semimoste/,,59.921376,30.296815,,528,False,attractions###interesting-places,14 февраля. лучшее###мосты###места для свидани...
5,27573,Интерактивный музей американских тачек Route 66,museum-route-66,"Конюшенная площадь, д. 2","пн–пт 12:00–21:00, сб, вс 11:00–21:00",<p>Интерактивный музей американских тачек Rout...,"<p>Museum Route 66 — это место, где оживает ам...",https://kudago.com/spb/place/museum-route-66/,http://museumroute66.ru/,59.937684,30.323180,Невский Проспект,51,False,museums,"музеи (раздел музеи, выставки)"
6,2375,Большой Санкт-Петербургский цирк,cirk-na-fontanke,"наб. Фонтанки, д. 3/А",касса: ежедневно 11:00–19:00,"<p>Большой Санкт-Петербургский цирк, известный...",<p>Большой государственный цирк в Санкт-Петерб...,https://kudago.com/spb/place/cirk-na-fontanke/,http://www.circus.spb.ru/ru/,59.938743,30.340913,Гостиный Двор,76,False,kids###circus,цирки (развлечения)###детские места (раздел де...
7,6875,Горчаковщинский водопад и Любшанская крепость,lyubshanskaya-krepost,"Ленинградская область, Волховский район, дерев...",,<p>Недалеко от исторической деревни Иссад в ле...,<p>Красивый лесной водопад и древнейшая крепос...,https://kudago.com/spb/place/lyubshanskaya-kre...,,60.017067,32.331795,,325,False,attractions###suburb,на свежем воздухе###пригороды (загородный отды...
8,6878,Танечкина пещера,peshera-tanechkina,"Ленинградская область, Волховский район, д. Ст...",,<p>В последнее десятилетие XIX века в этом рай...,"<p>Неподалёку от деревни Старая Ладога, на рек...",https://kudago.com/spb/place/peshera-tanechkina/,,60.011816,32.302895,,417,False,attractions,достопримечательности (интересные места)###пещеры
9,950,особняк Брусницыных,osobnyak-brusnicinih,"Кожевенная линия, д. 27",,<p>Судя по сохранившимся историческим документ...,"<p>Любители мистики знают наверняка, что знаме...",https://kudago.com/spb/place/osobnyak-brusnici...,,59.922769,30.249833,"Василеостровская, Приморская",545,False,attractions###houses###homesteads###photo-places,необычные дома###достопримечательности (интере...


In [127]:
places_table.to_csv("places.csv", sep=',', index=False, quoting=1)